## Algorithme d'encodage de texte pour Large Language Models : Byte Pair Encoding (BPE) et Tokenization

Pour celles et ceux d'entre vous qui ne disposent pas des droits d'accès pour utiliser les bibliothèques tierces requises pour ce TP (tiktoken et transformers), un environnement en ligne est disponible à l'URL https://mybinder.org/v2/gh/tpaviot/binderenv/HEAD?filepath=

Nous allons travailler à diviser un texte en briques de base connues sous le nom de "vocabulaire" et d'associer ainsi à une chapine de caractères une succession d'entiers. La tokenisation la plus élémentaire est celle consistant à associer à un mot l'ensemble des valeurs de la table ASCII (le vocabulaire contient $2^7=128$ briques de base) :
```
print([ord(c) for c in "Salut"])
[83, 97, 108, 117, 116]
```


Mais ceci n'est pas suffisant pour travailler avec les grands modèles de langage type ChatGPT.

Le **corpus** de texte est le jeu de données connu sous le nom de "Tiny shakespeare" accessible à l'url : https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Ce TP a été construit à partir, entre autres, des ressources suivantes disponibles en ligne, que vous êtes invité.e.s à prendre le temps de consulter (en particulier les vidéos de Karpathy) :

* La documenation HuggingFace (https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt)

* La chaîne vidéo YouTube d'Andrej Karpathy (https://www.youtube.com/@AndrejKarpathy/)

Ce **TP comporte trois parties** :
* dans une première partie, nous nous intéressons à un algorithme qui associe à chaque lettre un entier

* dans une deuxième partie, nous expérimentons un algorithme qui découpe les mots en token de 2 caractères

* dans une troisième partie, nous implémentons un algorithmes plus avancé appelé BPE qui permet d'encoder n'importe quelle chaine de caractères dans une liste d'entiers.

Pour chacun de ces trois algorithmes, nous comparons :
* la qualité, c'est-à-dire le nombre d'entiers requis pour encoder la chaîne. Plus ce nombre est petit, plus l'aglo est efficient

* le temps de calcul nécessaire pour encoder/décoder, étant entendu que, dans le domaine des modèles de langage, les texte à encoder peuvent être de plus giga octets. Plus le temps est court, plus l'algorithme est efficient.

Dans la suite, on appellera `token` un de ces motifs de base et `tokenization` le processus consistant à découper une chaîne de caractères en éléments de base disponibles dans un vocabulaire. On utilisera le terme *token* pour désigner sans distinction l'élément de base du vocabulaire ou l'entier associé, qui sont en bijection. L'`encoding` est le processus permettant de passer de la chaîne à la liste de tokens et donc d'entiers, le `decodage` le processus réciproque (passage d'une liste d'entiers à une chaîne de caractères).

## Question 1 - Chargement du jeu de données

Avec la commande `wget` directement dans ce notebook, télécharger le contenu du fichier `tinyshakespeare` et le stocker dans le répertoire courant.

In [ ]:
# votre code ici

# Première partie - Character Level Tokenization

## Question 2

* Charger le contenu du fichier dans une variable nommée `text`. Vous spécifierez un encodage de type utf-8 ;

* afficher les 200 premiers caractères du texte ;

* afficher le nombre total de caractères du texte.

In [ ]:
# votre code ici

## Question 3

Créer une fonction `build_vocab` qui prend comme paramètre une chaîne de carcatères `input_str` et qui renvoie :

* la liste  `chars` de tous les caractères utilisés, sans doublon, classée dans l'ordre des codes ASCII des caractères

* la taille `vocab_size` de cette liste

* vérifier avec un `assert` que pour la chaîne `"Andrej Karpathy, né le 23 octobre 1986, est un informaticien slovaco-canadien qui a été directeur de l'intelligence artificielle et du pilotage automatique chez Tesla. Il travaille actuellement pour OpenAI"` le résultat concaténé retourné est `" ',-.123689AIKOTabcdefghijlmnopqrstuvyzé"` et la longueur `40`.

In [ ]:
# votre code ici

## Question 4. Character Level Tokenization
Il s'agit de convertir ce texte en une séquence d'entiers à partir du vocabulaire défini précédemment.

* créer une fonction `encode` qui prend en paramètre une liste de caractères et renvoie la liste des indices des caractères correspondant dans la liste `vocab`

* créer une fonction `decode` qui est la fonction réciproque

* vérifier que `encode("hii there")` renvoie `[46, 47, 47, 1, 58, 46, 43, 56, 43]`

* vérifier que `decode([46, 47, 47, 1, 58, 46, 43, 56, 43])` renvoie `"hii_there"`

Dans cette question, nous avons associé, dans la fonction `encode`, un entier à chaque caractère, ce qui s'appelle `Character Level Tokenization`.

In [ ]:
# votre code ici

## Question 5. Performances de notre Character Encoder

* Mesurer le temps total nécessaire pour encoder le texte complet du texte tiny_shakespeare avec la fonction précédente ;

* Afficher la vitesse d'encodage en octets/secondes ;

* Mesurer et afficher le nombre d'éléments du texte encodé ;

* Mesurer le temps total de décodage pour le texte, et exprimer de la même manière la vitesse de décodage en octets/s.

In [ ]:
# votre code ici

## Question 6. Qualité de notre Character Encoder

* encoder le texte suivant: "Napoleon is a spectacle-filled action epic that details the checkered rise and fall of the iconic French Emperor Napoleon Bonaparte". Quelle est la taille de la liste obtenue ?
  
* encoder le texte suivant: "Napoléon est un film réalisé par Ridley Scott avec Joaquin Phoenix, Vanessa Kirby." Que constatez-vous ? quelle solution pouvez-vous apporter ?

In [ ]:
# votre code ici

# Partie 2

## Question 7. Introduction à la problématique de Subword tokenization
Nous pouvons travailler à partir d'un vocabulaire qui n'est pas constitué que de caractères simples mais de séquences de 2 caractères, ce qui permettra d'avoir des encodages plus courts.

Par exemple, si le vocabulaire est constitué des éléments: `vocab = ['ch', 'ien', 'at']` alors l'encodage du mot `chien` sera `[0, 1]` et celui du mot `chat` sera `[0, 2]`, ne prenant dans les deux cas que deux entiers alors qu'il en aurait fallu 4 avec la méthode des questions précédentes. Il s'agit dans ce cas d'un algorithme de "SubWord encoding", plus performant de toute évidence puisqu'il divise dans ce cas le nombre d'entiers par 2.

* définir une fonction `split_pair` qui prend une chaine de caractères et scinde la chaîne de caractères en groupes de deux caractères. Si la longueur de la chaîne de caractères est impaire alors la dernière lettre sera un caractère seul.

* vérifier que la fonction `split_pair` appliquée à la chaîne `"Napoleon"` renvoie `['Na', 'po', 'le', 'on']`

* vérifier que la fonction `split_pair` appliquée à la chaîne `"Napoleon3"`renvoie `['Na', 'po', 'le', 'on', '3']`

* comme dans la question 3, construire ensuite un vocabulaire à partir de cette liste de paires de caractères, sans doublons. Vérifier que pour la chaîne `"un chien et un chat rigolent, ha ha"` vous obtenez le vocabulaire :
`[' c', ' e', ' h', ', ', 'a', 'en', 'go', 'ha', 'hi', 'le', 'nt', 'ri', 't ', 'un']`
et une taille de vocabulaire de `14` éléments.
  

In [ ]:
# votre code ici

## Question 8. Un pair encoder simplifié

* créer le vocabulaire correspondant au jeu de données `tiny_shakespeare`. Vérifier que la taille du vocabulaire est de `1334` ;

* créer une fonction `encode_pair` et `decode_pair` qui s'appuient sur le vocabulaire précédent ;

* vérifier que  l'encoder renvoie pour la chaine `'I say unto you, what he hath done famously'` est `[391, 1165, 1296, 1237, 1208, 104, 1085, 156, 1267, 710, 88, 794, 887, 1203, 84, 1078, 794, 839, 1012, 1241, 993]`

* vérifier que le décodage renvoie la bonne chaine

In [ ]:
# votre code ici

## Question 9. Performances de ce pair encoder basique

* reprendre les mêmes questions que la question 5 pour mesurer les performances de l'encoder et du decoder en octets/s pour le texte complet tiney_shakespeare.

* conclure quant à la comparaison entre les deux encoders.

In [ ]:
# votre code ici

## Question 10. Limites de notre pair encoder

* Encoder la chaine "BUT Informatique de Nevers" avec le pair encoder précédent.

* Proposer et implémenter une solution pour corriger précédent pour le cas où les paires ne sont pas trouvées dans le vocabulaire.

* L'encoder modifié devra permettre d'obtenir la même longueur pour le tiny_shakespeare encodé, et proposer une solution pour n'importe quelle chaîne de caractères pour les caractères présents dans le texte original tiny_shakespeare.

* Vérifier l'impact de votre modification en termes de performances.

In [ ]:
# votre code ici

## Partie 3. Algorithme Byte Pair Encoding (BPE)


Dans l'exemple précédent, nous avons construit des paires de lettres de manière irréfléchie, simplement en stockant les paires au fur et à mesure qu'elles se présentent. L'algorithme Byte Pair Encoding permet de construire un vocabulaire de **token** (groupes de 2 ou plus lettres formant le vocabulaire de base) à partir de l'**analyse de la fréquence d'occurrence** dans un **corpus** (dans notre cas, le corpus est le fichier "tiny shakespeare"). Byte Pair Encoding (BPE) est un des algorithmes de **tokenization** les plus populaires, utilisé notamment dans les grands modèles de langage type ChatGPT.

Nous allons, dans les questions suivantes, implémenter un algorithme BPE à partir de zéro, puis ensuite nous le confronterons à des implémentations industrielles libres (celles d'OpenAI et HuggingFace).

## Question 11. Fréquence de mots

Ecrire une fonction `frequence_mots` qui prend comme paramètre une chaine de caractères `input_str` et qui renvoie un dictionnaire dont les clés sont les mots et les valeurs sont le nombre d'occurrences de ces mots dans la chaîne.

Par exemple, dans la chaîne "le chien Pluto et le chien Milou", le mot "chien" est présent 2 fois, le mot 'le' aussi, on obtiendra donc: 

{'le': 2, 'chien': 2, 'Pluto': 1, 'et': 1, 'Milou': 1)}

Nous allons travailler, dans ce qui suit, avec la chaîne de caractères suivante:
```python
corpus = "This is the Hugging Face Course. This chapter is about tokenization. This section shows several tokenizer algorithms. Hopefully, you will be able to understand how they are trained and generate tokens."
```

Vérifier que
```python
mots_freqs = frequence_mots(corpus)
print(mots_freqs)
```

renvoie bien

```python
{'This': 3, 'is': 2, 'the': 1, 'Hugging': 1, 'Face': 1, 'Course.': 1, 'chapter': 1, 'about': 1, 'tokenization.': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms.': 1, 'Hopefully,': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens.': 1}
```

In [ ]:
# votre code ici

## Question 12. Alphabet et vocabulaire

L'étape suivante est de déterminer le vocabulaire de base, formé par l'ensemble des caractères utilisés dans le corpus.

* Ecrire une fonction `calcule_alphabet` qui prend comme paramètre un dictionnaire de fréquences de mots `dict_freq` et qui renvoie la liste des lettres utilisées ;

* ensuite, créer le `vocabulaire` en ajoutant le token spécial `<|endoftext|>`:
```python
vocabulaire = ["<|endoftext|>"] + alphabet.copy()
```

Vous vérifierez que vous obtenez le vocabulaire suivant :
```python
['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']
```


In [ ]:
# votre code ici

## Question 13. Splits

Ecrire une fonction `calcule_splits` qui prend comme paramètre une liste de mots `liste_mots`et qui renvoie un dictionnaire qui associe à chaque mot la liste des lettres qui le composent. Par exemple :
```python
{'This': ['T', 'h', 'i', 's'], 'is': ['i', 's'], ...
```
Appliquer cette fonction aux mots servant de clé dans la dictionnaire `freqs`

In [ ]:
# votre code ici

## Question 14 - Fréquence des paires de lettres ou groupes de lettres

Il s'agit ensuite de déterminer la fréquence de l'occurrence de chaque paire de lettres. Par exemple, le mot 'This' est associé aux lettres 'T', 'h', 'i et 's'. Il faut chercher, dans tous les mots, le nombre d'occurrences de 'T', 'h', puis de 'h','i', et de 'i','s'. Et ainsi de suite pour chaque mot.

Ecrire une fonction `calcule_pair_freqs` qui retourne un dictionnaire avec comme clé un couple de lettres et comme valeur le nombre d'occurrences trouvées dans tous les mots. 

On aura par exemple en sortie:

```python
{('T', 'h'): 3,
 ('h', 'i'): 3,
 ('i', 's'): 5,
 ('t', 'h'): 3,
 ('h', 'e'): 2,
 ('H', 'u'): 1,
 ('u', 'g'): 1,
 ('g', 'g'): 1,
 ('g', 'i'): 1,
 ('i', 'n'): 2,
 ('n', 'g'): 1,
 ...}
```

In [ ]:
# votre code ici

## Question 15. Paire la plus fréquente

Créer une fonction `paire_la_plus_frequente` qui prend comme paramètre le dictionnaire issu de la fonction précédente (celui contenant la fréquence de chaque paire) et qui retourne la paire la plus fréquente du corpus ainsi que le nombre correspondant à la fréquence. Si deux paires présentent le même nombre d'occurrences, la fonction renvoie la première paire rencontrée dans le parcours de l'ensemble des paires.

Vérifier que `paire_la_plus_frequente(pair_freqs)` renvoie `(('i', 's'), 5)`

In [ ]:
# votre code ici

## Question 16. Extension du vocabulaire

On va maintenant rajouter au vocabulaire les combinaisons de lettres les plus fréquentes dans le texte.

* créer un dictionnaire `fusions` qui associe, à la paire précédente `('i', 's')` la paire concaténée `'is'`

* ajouter cette chaîne concaténée à la liste `vocabulaire`.

Remarque : cette question est très facile, inutile de créer une fonction.

Vous vérifierez que

```python
print(fusions)
print(vocabulaire)
```
renvoie
```
{('i', 's'): 'is'}
['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'is']
```

In [ ]:
# votre code ici

## Question 16. Fusion des paires

C'est la dernière étape de l'algorithme de création du vocabulaire. On se donne une taille maximale du vocabulaire `vocab_size` que l'on fixe à `50`. Reproduire l'étape précédente (reherche de la paire la plus fréquence, fusion, ajout au vocabulaire) jusqu'à ce que la taille maximale du vocabulaire soit atteinte.

Pour cette valeur de `vocab_size`, vérifier que vous obtenez le vocabulaire suivant :

```python
['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'is', 'is', 'er', 'to', 'en', 'Th', 'This', 'th', 'ou', 'se', 'tok', 'token', 'nd', 'the', 'in', 'ab', 'tokeni', 'tokeniz', 'at', 'io']
```

In [ ]:
# votre code ici

## Question 17 - Tokenization

La dernière étape de ce voyage vers les tokens consiste à créer une fonction `tokenize` qui prend une chaine de caractères et, comme dans le début de ce TP, contient l'ensemble des entiers faisant référence au vocabulaire de 50 termes construit précédemment.

Vérifier que la tokenization :

```python
print(tokenize('This is not a token.'))
```

Donne bien

```python
['This', 'is', 'n', 'o', 't', 'a', 'token', '.']
```

In [ ]:
# votre code ici

## Question 18 - Bilan de la qualité algorithmique

Pour la châine de caractères `This is not a token`, comparer la taille de la liste d'entiers obtenue pour chacun des 3 tokenizers étudiés.


## Question 19 - Performances de notre BPE

Reprendre les questions précédentes en travaillant à partir du corpus `tinyshakespeare`.

## Question 20. Implémentations industrielles de tokenizers GPT2 - OpenAI, HuggingFace

* BPE est utilisé par OpenAI pour ses gpt depuis gpt2. C'est la bibliothèque `tiktoken` (https://github.com/openai/tiktoken) qui implémente cet algorithme

* BPE est aussi utilisé par un autre grand acteur de l'IA générative : HuggingFace, dans sa bibliothèque `transformers` (https://github.com/huggingface/transformers)

Pour utiliser la bibliothèque **tiktoken**, encoder/décoder :
```python
enc = tiktoken.get_encoding('gpt2')
print(f"Nombre d'élements dans le vocabulaire : {enc.n_vocab}")
enc.encode('This is not a token')
```

Pour utiliser la bibliothèque **transformers**, encoder/décoder :
```python
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer("This is not a token")['input_ids']
```

* Quelle est la taille du vocabulaire `gpt2`?

* Vérifier que les deux implémentations renvoient la même liste d'entiers pour l'encodage de la chaîne `This is not a token`.

* Comparer ces implémentations de BPE avec celle que nous avons faite précédemment.

* Comparer ces deux bibliothèques en encodage/décodage par rapport à la vitesse en octets/seconde.

* Conclure.

In [ ]:
# votre code ici